In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os

## Data Loading and Preparation

In [ ]:
files=os.listdir(r"C:\Users\Jashank\New folder\uber-pickups-in-new-york-city/")[-7:]

In [ ]:
files.remove('uber-raw-data-janjune-15.csv')

In [ ]:
address=r"C:\Users\Jashank\New folder\uber-pickups-in-new-york-city"
final=pd.DataFrame()
for file in files:
    df=pd.read_csv(address+"/"+file,encoding="utf-8")
    final=pd.concat([df,final])

In [ ]:
df=final.copy()

In [ ]:
df.head()

In [ ]:
df.dtypes

##### Lat : The latitude of the Uber pickup

##### Lon : The longitude of the Uber pickup

##### Base : The TLC base company code affiliated with the Uber pickup

##### Latitude is the measurement of distance north or south of the Equator.

##### Every location on earth has a global address. Because the address is in numbers, people can communicate about location no matter what language they might speak. A global address is given as two numbers called coordinates. The two numbers are a location's latitude number and its longitude number ("Lat/Long").

In [ ]:
df["Date/Time"]=pd.to_datetime(df["Date/Time"],format="%m/%d/%Y %H:%M:%S")
df["Weekdays"]=df["Date/Time"].dt.day_name()
df['day']=df['Date/Time'].dt.day
df['minute']=df['Date/Time'].dt.minute
df['month']=df['Date/Time'].dt.month
df['hour']=df['Date/Time'].dt.hour

#### Analysis of journey by Week-days

In [ ]:
import plotly.express as px

In [ ]:
px.bar(x=df['Weekdays'].value_counts().index,
      y=df['Weekdays'].value_counts().values, labels={"x":"Days","y":"Total Journey"}
      )

#### seems to have highest sales on Thursday

### Analysis by Hour

In [ ]:
px.histogram(df['hour'])

#### It peaks during evening time when people are logging off from work

### Analysis by Hour on each month

In [ ]:

plt.figure(figsize=(40,20))
for i,month in enumerate(df['month'].unique()):
    plt.subplot(3,2,i+1)
    df[df['month']==month]['hour'].hist()


#### Analysis of Journey of Each  Day

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(df['day'], bins=30, rwidth=.8, range=(0.5, 30.5))
plt.xlabel('date of the month')
plt.ylabel('Total Journeys')
plt.title('Journeys by Month Day')

### getting Rush in hour 

In [ ]:
ax=sns.pointplot(x="hour",y="Lat", hue="Weekdays",data=df)
ax.set_title('hoursoffday vs latiitide of passenger')

In [ ]:
df_out=df[df['Weekdays']=='Sunday']

In [ ]:
df_out.groupby(['Lat','Lon'])['Weekdays'].count().reset_index()

In [ ]:
from folium.plugins import HeatMap
import folium
from folium.plugins import HeatMap
basemap=folium.Map()

In [ ]:
HeatMap(df_out.groupby(['Lat','Lon'])['Weekdays'].count().reset_index(),zoom=20,radius=15).add_to(basemap)
basemap

##### Lets create a function for a specific day

In [ ]:
def plot(df,day):
    df_out=df[df['Weekdays']==day]
    df_out.groupby(['Lat','Lon'])['Weekdays'].count().reset_index()
    HeatMap(df_out.groupby(['Lat','Lon'])['Weekdays'].count().reset_index(),zoom=20,radius=15).add_to(basemap)
    return basemap


In [ ]:
plot(df,'Sunday')

##### Uber pickups by the month in NYC

In [ ]:
rush_15=pd.read_csv(r"C:\\Users\Jashank\\New folder\\uber-pickups-in-new-york-city\\uber-raw-data-janjune-15.csv")


In [ ]:
rush_15["Pickup_date"]=pd.to_datetime(rush_15["Pickup_date"],format="%Y-%m-%d %H:%M:%S")

In [ ]:
rush_15["Weekdays"]=rush_15["Pickup_date"].dt.day_name()
rush_15["Day"]=rush_15["Pickup_date"].dt.day
rush_15["month"]=rush_15["Pickup_date"].dt.month
rush_15["hour"]=rush_15["Pickup_date"].dt.hour
rush_15["min"]=rush_15["Pickup_date"].dt.minute

In [ ]:
px.bar(x=rush_15['month'].value_counts().index,
           y=rush_15['month'].value_counts().values)

#### We can see that the number of Uber pickup has been steadily increasing throughout the first half of 2015 in NYC

#### Analysing Rush in New york City

In [ ]:
ax=sns.countplot(rush_15['hour'])


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
s=rush_15.groupby(["Weekdays","hour"])["Pickup_date"].count().reset_index()
s.columns=["Weekdays","hour","count"]
plt.figure(figsize=(12,8))
sns.pointplot(data=s,x="hour",y="count",hue="Weekdays")